In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

import captum

import torch
import torchtext
import torchtext.data

import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
from torchtext.data.utils import get_tokenizer

from models.cnn.model import CNN
from datasets.scar import SCAR


tokenizer = get_tokenizer('basic_english')

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Survival 6 months
ckpt_name = "CNN_20230101-1542"
ckpt_path = os.path.join(r"C:\Users\jjnunez\PycharmProjects\scar_nlp_survival\results\paper_revision\surv_mo_6\CNN", ckpt_name + ".pt")

checkpoint = torch.load(ckpt_path)

model = CNN(config=checkpoint['config'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model = model.to(device)
print(model)

self.embed num_embeddings is: 22338 while embedding_dim is 300
CNN(
  (embed): Embedding(22338, 300)
  (conv1): Conv2d(1, 500, kernel_size=(3, 300), stride=(1, 1), padding=(2, 0))
  (conv2): Conv2d(1, 500, kernel_size=(4, 300), stride=(1, 1), padding=(3, 0))
  (conv3): Conv2d(1, 500, kernel_size=(5, 300), stride=(1, 1), padding=(4, 0))
  (dropout): Dropout(p=0.8, inplace=False)
  (fc1): Linear(in_features=1500, out_features=1, bias=True)
)


In [4]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

In [5]:
# from torchtext import vocab

#loaded_vectors = vocab.GloVe(name='6B', dim=100)

#TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
    
#TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
#Label.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))

#print('Vocabulary Size: ', len(TEXT.vocab))
#print('Vocabulary Size: ', len(Label.vocab))
#print(train)
#print(Label.vocab.itos)
config = checkpoint['config']
scar = SCAR(config.batch_size, config.data_dir, config.target)



In [6]:
vocab = scar.vocab
itos = vocab.get_itos()
print(len(vocab))

22338


In [7]:
# PAD_IND = TEXT.vocab.stoi[TEXT.pad_token]
print(vocab['doctor'])
PAD_IND = vocab['<PAD>']
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embed)

909


In [8]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 1100, label = 0):
    text = [tok for tok in tokenizer(sentence.lower())]
    if len(text) < min_len:
        text += ['<PAD>'] * (min_len - len(text))
    indexed = [vocab[t] for t in text]

    model.zero_grad()
   
    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)
    
    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
    
    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=500, return_convergence_delta=True
                                          )
    # Replace Label with Text below
    print(itos[pred_ind])
    print('pred: ', itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            itos[pred_ind],
                            itos[label],
                            itos[1],
                            attributions.sum(),
                            text,
                            delta))

In [9]:
# interpret_sentence(model, text, label=1)

#file = open('anon.txt',mode='r')
file = open('anon_extra_3.txt',mode='r')
text = file.read()
file.close()

interpret_sentence(model, text, label=1)


C:\Users\jjnunez\.conda\envs\scar_nlp\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


here is the single layer: Embedding(22338, 300)
line 294 if FALSE
entered hook_wrapper
entered registered forward_hooks
handle: <torch.utils.hooks.RemovableHandle object at 0x0000004DF61D8D30>
about to run_forward
Entered run_forward
in forward_run, len(forward_func_args) != 0
entering forward_func: CNN(
  (embed): Embedding(22338, 300)
  (conv1): Conv2d(1, 500, kernel_size=(3, 300), stride=(1, 1), padding=(2, 0))
  (conv2): Conv2d(1, 500, kernel_size=(4, 300), stride=(1, 1), padding=(3, 0))
  (conv3): Conv2d(1, 500, kernel_size=(5, 300), stride=(1, 1), padding=(4, 0))
  (dropout): Dropout(p=0.8, inplace=False)
  (fc1): Linear(in_features=1500, out_features=1, bias=True)
)
entered forward_hook
if FALSE, here is the saved_layer: defaultdict(<class 'dict'>, {})
exiting forward_func
finished run_forward
about to check length of saved_layer
saved_layer length checked and passed!
here is the single layer: Embedding(22338, 300)
line 294 if FALSE
entered hook_wrapper
entered registered forwar

In [10]:
print('Visualize attributions based on Integrated Gradients')
_ = visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients
